In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialisation de Spark
spark = SparkSession.builder \
    .appName("Évaluation Modèle GBT") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .enableHiveSupport() \
    .getOrCreate()

# Charger les données et réduire les partitions pour optimiser le traitement
df = spark.sql("SELECT * FROM concessionnaire.client_immat_with_catk9").repartition(200)

# Charger le modèle sauvegardé
model = PipelineModel.load("hdfs://namenode:9000/models/ovr_gbt_modelk9")

# Indexation de la colonne `categorie` si nécessaire
if "indexed_categorie" not in df.columns:
    indexer = StringIndexer(inputCol="categorie", outputCol="indexed_categorie", handleInvalid="keep")
    df = indexer.fit(df).transform(df)

# Prédictions
predictions = model.transform(df)

# Évaluation
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexed_categorie", predictionCol="ovr_prediction", metricName="weightedFMeasure"
)
f1_score = evaluator.evaluate(predictions)
print(f"F1 Score Pondéré : {f1_score:.4f}")

# Fin de la session Spark
spark.stop()


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 40116)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

Py4JError: An error occurred while calling o2219.evaluate